In [ ]:
import pandas as pd
import numpy as np
import skbio
from skbio import TreeNode
import dendropy

In [4]:
from sklearn.decomposition import PCA

In [5]:
import torch.nn as nn

In [6]:
import biom
from biom.util import biom_open
import torch
from tqdm import tqdm

In [7]:
import numpy as np
import pandas as pd
from skbio import TreeNode
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list

In [8]:
from sklearn.model_selection import KFold

In [9]:
import matplotlib.pyplot as plt

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [12]:
def gaussian_kernel(d, rho=2):
    """
    weight = exp(-2 * ρ * d^2)
    """
    return np.exp(-2 * rho * d ** 2)

In [ ]:
def build_graph_from_sample(otu_list, abundance_list, label, sample_id, full_tree):

    leaf_abundance_map = {otu: abund for otu, abund in zip(otu_list, abundance_list)}
    
    # tree_sample = full_tree.shear(otu_list)
    tree_sample = full_tree
    all_nodes_in_tree = list(tree_sample.traverse(self_before=True))
    node_to_idx = {node.name: i for i, node in enumerate(all_nodes_in_tree)}

    full_abundance_map = {}
    x = torch.zeros((len(all_nodes_in_tree), 1), dtype=torch.float)
    edge_list, edge_weight_list = [], []
    node_id = []
    for node in tree_sample.postorder():
        node_id.append(node_to_idx[node.name])
        if node.is_tip():
            full_abundance_map[node.name] = leaf_abundance_map.get(node.name, 0.0)
            x[node_to_idx[node.name]] = full_abundance_map[node.name]
        else:
            child_abundances = [full_abundance_map.get(child.name, 0.0) for child in node.children]
            full_abundance_map[node.name] = sum(child_abundances)
            x[node_to_idx[node.name]] = full_abundance_map[node.name]
            
        if not node.is_root():
            node_idx = node_to_idx[node.name]
            parent_idx = node_to_idx[node.parent.name]
            parent_idx = node_to_idx[node.parent.name]
            edge_list.extend([[parent_idx, node_idx], [node_idx, parent_idx]])
            distance = node.length if node.length is not None else 0.0
            weight = gaussian_kernel(distance, rho=5e-5) 
            edge_weight_list.extend([weight, weight])
            
    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    y = torch.tensor([label], dtype=torch.long)
    x = torch.tensor(x, dtype=torch.float)
    edge_attr = torch.tensor(edge_weight_list, dtype=torch.float).view(-1, 1)
    node_id = torch.tensor(node_id, dtype=torch.long)
    
    data = Data(x=x, node_id=node_id, edge_index=edge_index, edge_attr=edge_attr, y=y, sample_id=sample_id)

    return data, x, edge_index, edge_attr, node_to_idx

### IBD 16S

In [ ]:
metadata = pd.read_csv("../data/IBD_16S/metadata.tsv", sep="\t", index_col=0)

In [ ]:
table = biom.load_table("../data/IBD_16S/table.biom")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    table_train.remove_empty()
    with biom.util.biom_open(f"../data/IBD_16S/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    table_test.remove_empty()
    with biom.util.biom_open(f"../data/IBD_16S/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"../data/IBD_16S/data/Real_Dateset_16S_IBD/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"../data/IBD_16S/data/Real_Dateset_16S_IBD/data/test_{n}.csv")
        
    n = n + 1

### CRC 16S

In [1028]:
table = pd.read_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/16S_CRC.csv")

metadata = pd.DataFrame({"sample_id": table.iloc[:, 0].values, "group": table["Group"].values})
metadata["group"] = [0 if i == "HC" else 1 for i in metadata["group"].values]
metadata.to_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/metadata.tsv", sep="\t", index=False)

table = biom.Table(data=table.iloc[:, 1:-1].values.T, observation_ids=table.iloc[:, 1:-1].columns.values, sample_ids=table.iloc[:, 0].values)
with biom.util.biom_open("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

metadata = metadata.set_index("sample_id")
table = biom.load_table("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/table.biom")

kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/test_{n}.csv")
        
    n = n + 1

In [334]:
for i in range(1, 6):
    table_train = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/train_{i}.biom")
    table_test = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/test_{i}.biom")
    for m in ["without_low", "without_high"]:
        for n in [0.1, 0.2, 0.4, 0.5, 0.6, 0.8]:
            tree = TreeNode.read(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/phylogeny_{m}_{n}.nwk")
            leaf_names = [node.name for node in tree.tips()]
            
            table_train_pick = table_train.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/train_{m}_{n}_{i}.biom", 'w') as f:
                table_train_pick.to_hdf5(f, 'example')
                
            table_test_pick = table_test.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/data/test_{m}_{n}_{i}.biom", 'w') as f:
                table_test_pick.to_hdf5(f, 'example')

### CRC WGS

In [1031]:
table = pd.read_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/WGS_CRC.csv", sep="\t")

metadata = pd.DataFrame({"sample_id": table.iloc[:, 0].values, "group": table["Group"].values})
metadata["group"] = [0 if i == "CTR" else 1 for i in metadata["group"].values]
metadata.to_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/metadata.tsv", sep="\t", index=False)

fid = table.iloc[:, 1:-1].columns.values.astype(str)
fid = np.char.replace(fid, '__', '.')
fid = np.char.replace(fid, '_', '.')
table = biom.Table(data=table.iloc[:, 1:-1].values.T, observation_ids=fid, sample_ids=table.iloc[:, 0].values)
with biom.util.biom_open("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

metadata = metadata.set_index("sample_id")
table = biom.load_table("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/table.biom")

kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/test_{n}.csv")
        
    n = n + 1

In [325]:
for i in range(1, 6):
    table_train = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/train_{i}.biom")
    table_test = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/test_{i}.biom")
    for m in ["without_low", "without_high"]:
        for n in [0.1, 0.2, 0.4, 0.5, 0.6, 0.8]:
            tree = TreeNode.read(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/phylogeny_{m}_{n}.nwk")
            leaf_names = [node.name for node in tree.tips()]
            
            table_train_pick = table_train.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/train_{m}_{n}_{i}.biom", 'w') as f:
                table_train_pick.to_hdf5(f, 'example')
                
            table_test_pick = table_test.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/data/test_{m}_{n}_{i}.biom", 'w') as f:
                table_test_pick.to_hdf5(f, 'example')

### T2D WGS

In [1043]:
table = pd.read_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/WGS_T2D.csv")

metadata = pd.DataFrame({"sample_id": table.iloc[:, 0].values, "group": table["Group"].values})
metadata["group"] = [0 if i == "Health" else 1 for i in metadata["group"].values]
metadata.to_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/metadata.tsv", sep="\t", index=False)

fid = table.iloc[:, 1:-1].columns.values.astype(str)
fid = np.char.replace(fid, '__', '.')
fid = np.char.replace(fid, '_', '.')
table = biom.Table(data=table.iloc[:, 1:-1].values.T, observation_ids=fid, sample_ids=table.iloc[:, 0].values)
with biom.util.biom_open("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

metadata = metadata.set_index("sample_id")
table = biom.load_table("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/table.biom")

kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/test_{n}.csv")
        
    n = n + 1

In [331]:
for i in range(1, 6):
    table_train = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/train_{i}.biom")
    table_test = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/test_{i}.biom")
    for m in ["without_low", "without_high"]:
        for n in [0.1, 0.2, 0.4, 0.5, 0.6, 0.8]:
            tree = TreeNode.read(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/phylogeny_{m}_{n}.nwk")
            leaf_names = [node.name for node in tree.tips()]
            
            table_train_pick = table_train.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/train_{m}_{n}_{i}.biom", 'w') as f:
                table_train_pick.to_hdf5(f, 'example')
                
            table_test_pick = table_test.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/data/test_{m}_{n}_{i}.biom", 'w') as f:
                table_test_pick.to_hdf5(f, 'example')

### Dietary fiber

In [277]:
metadata = pd.read_excel("/home/dongbiao/word_embedding_microbiome/programe_test/dietary_fber/data/metadata.xls", index_col=0)
table = biom.load_table("/home/dongbiao/word_embedding_microbiome/programe_test/dietary_fber/data/projects/table_6721_2378.biom")
metadata = metadata.loc[table.ids(axis="sample")]
metadata = metadata.loc[metadata.study.values != "PRJEB2165"]
metadata = metadata.loc[metadata.study.values != "PRJNA385004"]
table = table.filter(metadata.index.values)
table.remove_empty()

6693 x 2270 <class 'biom.table.Table'> with 313231 nonzero entries (2% dense)

In [278]:
fid = table.ids(axis="observation")
prevalence = table.nonzero_counts(axis="observation")
table = table.filter(fid[prevalence > 50], axis='observation', inplace=False)
table.remove_empty()

930 x 2270 <class 'biom.table.Table'> with 252062 nonzero entries (11% dense)

In [279]:
metadata = metadata.loc[table.ids(axis="sample")]
table = table.norm(axis='sample', inplace=False)

In [280]:
with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

In [281]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{n}.csv")
        
    n = n + 1

In [324]:
for i in range(1, 6):
    table_train = biom.load_table(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{i}.biom")
    table_test = biom.load_table(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{i}.biom")
    for m in ["without_low", "without_high"]:
        for n in [0.1, 0.2, 0.4, 0.5, 0.6, 0.8]:
            tree = TreeNode.read(f"/home/dongbiao/GCN/data/dietary_fiber/phylogeny_{m}_{n}.nwk")
            leaf_names = [node.name for node in tree.tips()]
            
            table_train_pick = table_train.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{m}_{n}_{i}.biom", 'w') as f:
                table_train_pick.to_hdf5(f, 'example')
                
            table_test_pick = table_test.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{m}_{n}_{i}.biom", 'w') as f:
                table_test_pick.to_hdf5(f, 'example')

In [250]:
study_id = metadata.study.unique()
study_id

array(['PRJEB41443', 'PRJNA560950', 'PRJNA780023', 'PRJNA891951',
       'PRJNA293971', 'PRJNA306884', 'PRJNA428736'], dtype=object)

In [251]:
# PRJEB41443
temp = metadata.loc[metadata.study == study_id[0]]
sid_1 = temp.loc[temp.timepoint_numeric == 1].index.values
sid_2 = temp.loc[temp.group == 1].index.values
PRJEB41443_sid = list(sid_1) + list(sid_2)

In [252]:
# PRJNA560950
metadata.loc[metadata.study == study_id[1]].groupby(["subject_id", "group", "timepoint_id"]).count()
temp = metadata.loc[metadata.study == study_id[1]]
PRJNA560950_sid = temp.loc[temp.timepoint_id == "W4"].index.tolist()

In [253]:
# PRJNA780023
temp = metadata.loc[metadata.study == study_id[2]]
sid_1 = temp.loc[temp.group == 0].drop_duplicates(subset='subject_id').index.values
sid_2 = temp.loc[temp.group == 1].drop_duplicates(subset='subject_id').index.values
PRJNA780023_sid = list(sid_1) + list(sid_2) 

In [254]:
# PRJNA891951
temp = metadata.loc[metadata.study == study_id[3]]
PRJNA891951_sid = temp.loc[temp.timepoint == "after"].index.tolist()

In [255]:
# PRJNA293971
temp = metadata.loc[metadata.study == study_id[4]]
PRJNA293971_sid = temp.loc[[i in ["before_1", "after_3"] for i in temp.timepoint_id.values]].index.tolist()

In [256]:
# PRJNA306884
temp = metadata.loc[metadata.study == study_id[5]]
PRJNA306884_sid = temp.loc[[i in [4, 8] for i in temp.timepoint_numeric.values]].index.tolist()

In [257]:
# PRJNA428736
temp = metadata.loc[metadata.study == study_id[6]]
sid_1 = temp.loc[temp.group == 0].drop_duplicates(subset='subject_id').index.values
sid_2 = temp.loc[temp.group == 1].drop_duplicates(subset='subject_id').index.values
PRJNA428736_sid = list(sid_1) + list(sid_2) 

In [258]:
pick_sid = PRJEB41443_sid + PRJNA560950_sid + PRJNA780023_sid + PRJNA891951_sid + PRJNA293971_sid + PRJNA306884_sid + PRJNA428736_sid
table.filter(pick_sid, axis="sample")
table.remove_empty()

6235 x 731 <class 'biom.table.Table'> with 105646 nonzero entries (2% dense)

In [259]:
fid = table.ids(axis="observation")
prevalence = table.nonzero_counts(axis="observation")
table = table.filter(fid[prevalence > 20], axis='observation', inplace=False)
table.remove_empty()

936 x 731 <class 'biom.table.Table'> with 80180 nonzero entries (11% dense)

In [260]:
with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

In [261]:
metadata = metadata.loc[table.ids(axis="sample")]

In [262]:
table = table.norm(axis='sample', inplace=False)

In [263]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{n}.csv")
        
    n = n + 1

In [308]:
for i in range(1, 6):
    table_train = biom.load_table(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{i}.biom")
    table_test = biom.load_table(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{i}.biom")
    for m in ["without_low", "without_high"]:
        for n in [0.1, 0.2, 0.4, 0.5, 0.6, 0.8]:
            tree = TreeNode.read(f"/home/dongbiao/GCN/data/dietary_fiber/phylogeny_{m}_{n}.nwk")
            leaf_names = [node.name for node in tree.tips()]
            
            table_train_pick = table_train.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/train_{m}_{n}_{i}.biom", 'w') as f:
                table_train_pick.to_hdf5(f, 'example')
                
            table_test_pick = table_test.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/GCN/data/dietary_fiber/data/test_{m}_{n}_{i}.biom", 'w') as f:
                table_test_pick.to_hdf5(f, 'example')

### OSCC

In [1046]:
table = biom.load_table("/home/dongbiao/word_embedding_microbiome/all_data/oral/global-oral-microbiome/data/table_gg2_id99_24409_56564_v5.biom")

In [1047]:
metadata = pd.read_csv("/home/dongbiao/GCN/data/OSCC/metadata.txt", sep="\t", index_col=0, low_memory=False)
metadata = metadata.loc[metadata.diagnose != "OPMD"]
metadata.loc[:, "group"] = [1 if i == "OSCC" else 0 for i in metadata.diagnose.values]

In [1048]:
table_filter = None  
for i, study_id in enumerate(metadata.project_ID.unique()):
    metadata_study = metadata.loc[metadata.project_ID.values == study_id]
    sid = np.intersect1d(metadata_study.index.values, table.ids(axis="sample"))
    temp = table.filter(sid, inplace=False)
    temp.remove_empty()
    
    fid = temp.ids(axis="observation")
    prevalence = temp.nonzero_counts(axis="observation")
    temp = temp.filter(fid[prevalence > 10], axis='observation', inplace=False)
    temp.remove_empty()
    
    if table_filter is None:
        table_filter = temp
    else:
        table_filter = table_filter.merge(temp)

In [1049]:
table_filter

864 x 631 <class 'biom.table.Table'> with 88061 nonzero entries (16% dense)

In [1050]:
otu_retained, otu_removed = filter_low_abundance_asvs(table_filter.norm(axis='sample', inplace=False).to_dataframe())
table_filter = table_filter.filter(otu_retained, axis='observation', inplace=False)
table_filter.remove_empty()

569 x 631 <class 'biom.table.Table'> with 79776 nonzero entries (22% dense)

In [1051]:
with biom.util.biom_open("/home/dongbiao/GCN/data/OSCC/table.biom", 'w') as f:
    table_filter.to_hdf5(f, 'example')

In [862]:
metadata = metadata.loc[table_filter.ids(axis="sample")]
metadata.to_csv("/home/dongbiao/GCN/data/OSCC/metadata.tsv", sep="\t")

In [1052]:
metadata = pd.read_csv("/home/dongbiao/GCN/data/OSCC/metadata.tsv", sep="\t", index_col=0)
table = biom.load_table("/home/dongbiao/GCN/data/OSCC/table.biom")

In [1053]:
# study_id = metadata.project_ID.unique()
# n = 1
# for i in study_id:
#     sid = metadata.loc[metadata.project_ID != i].index.values
#     table_train = table.filter(sid, axis="sample", inplace=False)
#     with biom.util.biom_open(f"/home/dongbiao/GCN/data/OSCC/data/train_{n}.biom", 'w') as f:
#         table_train.to_hdf5(f, 'example')
#     sid = metadata.loc[metadata.project_ID == i].index.values
#     table_test = table.filter(sid, axis="sample", inplace=False)
#     with biom.util.biom_open(f"/home/dongbiao/GCN/data/OSCC/data/test_{n}.biom", 'w') as f:
#         table_test.to_hdf5(f, 'example')

#     table_train = table_train.norm(axis='sample', inplace=False).to_dataframe().T
#     table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
#     table_train.to_csv(f"/home/dongbiao/GCN/data/OSCC/data/train_{n}.csv")
#     table_test = table_test.norm(axis='sample', inplace=False).to_dataframe().T
#     table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
#     table_test.to_csv(f"/home/dongbiao/GCN/data/OSCC/data/test_{n}.csv")
    
#     n = n + 1
kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/OSCC/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/OSCC/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/GCN/data/OSCC/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/GCN/data/OSCC/data/test_{n}.csv")
        
    n = n + 1

### Multi-status classsification

In [53]:
table = pd.read_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/Multi-classification.csv")
metadata = pd.DataFrame({"sample_id": table.iloc[:, 0].values, "disease": table["Group"].values,
                         "group": encoder.fit_transform(table["Group"].values)})
metadata.to_csv("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/metadata.tsv", sep="\t", index=False)

fid = table.iloc[:, 1:-1].columns.values.astype(str)
table = biom.Table(data=table.iloc[:, 1:-1].values.T, observation_ids=fid, sample_ids=table.iloc[:, 0].values)
with biom.util.biom_open("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

metadata = metadata.set_index("sample_id")
table = biom.load_table("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/table.biom")

kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/test_{n}.csv")
        
    n = n + 1

In [337]:
for i in range(1, 6):
    table_train = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/train_{i}.biom")
    table_test = biom.load_table(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/test_{i}.biom")
    for m in ["without_low", "without_high"]:
        for n in [0.1, 0.2, 0.4, 0.5, 0.6, 0.8]:
            tree = TreeNode.read(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/phylogeny_{m}_{n}.nwk")
            leaf_names = [node.name for node in tree.tips()]
            
            table_train_pick = table_train.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/train_{m}_{n}_{i}.biom", 'w') as f:
                table_train_pick.to_hdf5(f, 'example')
                
            table_test_pick = table_test.filter(leaf_names, axis="observation", inplace=False)
            with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/data/test_{m}_{n}_{i}.biom", 'w') as f:
                table_test_pick.to_hdf5(f, 'example')

### Synthetic data

In [225]:
class GreengenesSyntheticGenerator:
    def __init__(self, tree_path, n_features=60, random_seed=42):
        """
        初始化生成器：基于真实的 Greengenes 树文件。
        """
        self.n_features = n_features
        self.rng = np.random.default_rng()
        
        print(f"正在加载进化树: {tree_path} ...")
        self.full_tree = TreeNode.read(tree_path)
        
        # 随机选择 OTU
        all_tips = [node.name for node in self.full_tree.tips()]
            
        self.selected_otus = self.rng.choice(all_tips, size=n_features, replace=False)
        self.leaf_names = list(self.selected_otus)
        
        # 剪切树并计算距离
        print("正在计算系统发育距离矩阵...")
        self.sheared_tree = self.full_tree.shear(self.leaf_names)
        dm = self.sheared_tree.tip_tip_distances()
        
        self.phylo_dist_matrix = np.zeros((n_features, n_features))
        for i, otu1 in enumerate(self.leaf_names):
            for j, otu2 in enumerate(self.leaf_names):
                self.phylo_dist_matrix[i, j] = dm[otu1, otu2]

        # =================================================================
        # 【修改重点】：强制平衡划分 Clade 1 和 Clade 2
        # =================================================================
        # 1. 使用 'ward' 方法进行聚类，它倾向于产生大小更均匀的簇（相比 average/single）
        #    注意：虽然 UPGMA (average) 更符合生物学进化，但为了模拟平衡数据，Ward 更稳健
        Z = linkage(squareform(self.phylo_dist_matrix), method='ward')
        
        # 2. 获取按树结构排序后的叶子节点索引列表
        #    leaves_list 返回的顺序保证了相邻的索引在树上是亲缘关系最近的
        ordered_indices = leaves_list(Z)
        
        # 3. 强制一分为二 (例如 60 -> 30 + 30)
        #    这样既保证了数量平衡，又因为基于树排序，保证了组内的系统发育相似性
        midpoint = len(ordered_indices) // 2
        clade1_indices = ordered_indices[:midpoint]
        clade2_indices = ordered_indices[midpoint:]
        
        # 4. 生成标签数组
        self.cluster_labels = np.zeros(n_features, dtype=int)
        self.cluster_labels[clade1_indices] = 1
        self.cluster_labels[clade2_indices] = 2
        
        print(f"聚类完成: Clade 1 ({len(clade1_indices)} 个), Clade 2 ({len(clade2_indices)} 个)")

    def _generate_base_abundance(self, n_samples, pattern_type):
        data = np.zeros((n_samples, self.n_features))
        
        clade1_idx = np.where(self.cluster_labels == 1)[0]
        clade2_idx = np.where(self.cluster_labels == 2)[0]
        
        if pattern_type == 'S1':
            # S1 主要在 Clade 1 (Cluster 1)
            active_idx = clade1_idx
            # if len(clade2_idx) > 0:
            #     active_idx = np.concatenate([active_idx, self.rng.choice(clade2_idx, size=int(len(clade2_idx)*0.1))])
        else:
            # S2 主要在 Clade 2 (Cluster 2)
            active_idx = clade2_idx
            # if len(clade1_idx) > 0:
            #     active_idx = np.concatenate([active_idx, self.rng.choice(clade1_idx, size=int(len(clade1_idx)*0.1))])
            
        for i in range(n_samples):
            base_vals = self.rng.dirichlet(np.ones(len(active_idx)), size=1)[0]
            data[i, active_idx] = base_vals
        return data

    def generate_dataset_1(self):
        """
        生成 Synthetic data set 1
        修改说明：
        - S3 (从 S1 扰动而来): 被扰动的特征从 S2 的主要特征 (Clade 2) 中选取。
        - S4 (从 S2 扰动而来): 被扰动的特征从 S1 的主要特征 (Clade 1) 中选取。
        """
        n_total_group1 = 77
        n_total_group2 = 71
        
        # 1. 生成基础数据
        data_g1 = self._generate_base_abundance(n_total_group1, 'S1')
        data_g2 = self._generate_base_abundance(n_total_group2, 'S2')
        
        labels_g1 = np.zeros(n_total_group1)
        labels_g2 = np.ones(n_total_group2)
        
        # 2. 添加噪声
        data_g1 += self.rng.normal(0, 1e-6, data_g1.shape)
        data_g2 += self.rng.normal(0, 1e-6, data_g2.shape)
        
        # 获取 S1 (Clade 1) 和 S2 (Clade 2) 对应的特征索引
        # self.cluster_labels 在 __init__ 中已经生成
        clade1_idx = np.where(self.cluster_labels == 1)[0]
        clade2_idx = np.where(self.cluster_labels == 2)[0]

        # 3. 引入特征错位 (Feature Misalignment)
        # 修改：增加 target_pool 参数，用于指定从哪个集合中选取要被扰动的特征
        def apply_misalignment(data, target_pool, move_id):
            n_samples = data.shape[0]
            # 随机选择 20%-30% 的样本进行扰动 (根据你的上一段代码逻辑调整回论文常见比例，或者保持你设定的0.8)
            # 这里我保持你代码中的 0.8-0.9，如果需要论文原比例请改为 0.2-0.3
            perturb_ratio = 0.90
            n_perturb = int(n_samples * perturb_ratio)
            perturb_indices = self.rng.choice(n_samples, n_perturb, replace=False)

            for idx in perturb_indices:
                # 随机选择一定比例的 OTU 进行移动
                otu_ratio = self.rng.uniform(0.45, 0.45)
                # 计算需要移动的特征数量
                actual_n_move = int(len(move_id) * otu_ratio)
                source_otus = self.rng.choice(move_id, actual_n_move, replace=False)
                replace_otus = self.rng.choice(target_pool, actual_n_move, replace=False)

                data[idx, replace_otus] = data[idx, source_otus]
                data[idx, source_otus] = 0

            return data

        # S1 -> S3: 扰动 S2 的特征 (即从 clade2_idx 中选特征来捣乱)
        # 逻辑：S1 样本中本来 S2 特征很少，现在我们特意去动这些特征(如果存在的话)
        data_g1 = apply_misalignment(data_g1, target_pool=clade2_idx, move_id=clade1_idx)
        
        # S2 -> S4: 扰动 S1 的特征 (即从 clade1_idx 中选特征来捣乱)
        data_g2 = apply_misalignment(data_g2, target_pool=clade1_idx, move_id=clade2_idx)
        
        # 非负截断
        data_g1 = np.maximum(data_g1, 0)
        data_g2 = np.maximum(data_g2, 0)
        
        X = np.vstack([data_g1, data_g2])
        y = np.concatenate([labels_g1, labels_g2])
        
        return X, y, self.leaf_names

In [604]:
tree_path = "/home/dongbiao/GCN/data/synthetic_data/phylogeny.nwk"

generator = GreengenesSyntheticGenerator(tree_path, n_features=200)

X, y, otu_ids = generator.generate_dataset_1()

正在加载进化树: /home/dongbiao/GCN/data/synthetic_data/phylogeny.nwk ...
正在计算系统发育距离矩阵...
聚类完成: Clade 1 (100 个), Clade 2 (100 个)


In [605]:
s = [f"S_{i}" for i in range(1, X.shape[0]+1)]

In [606]:
table = biom.Table(data=X.T, observation_ids=otu_ids, sample_ids=s)
with biom.util.biom_open("/home/dongbiao/GCN/data/synthetic_data/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

In [607]:
metadata = pd.DataFrame({"sample_id": s, "group": y.astype(np.int32)})
metadata.to_csv("/home/dongbiao/GCN/data/synthetic_data/metadata.txt", sep="\t", index=False)

In [608]:
metadata = metadata.set_index("sample_id")

In [609]:
table = biom.load_table("/home/dongbiao/GCN/data/synthetic_data/table.biom")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    table_train.remove_empty()
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/synthetic_data/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    table_test.remove_empty()
    with biom.util.biom_open(f"/home/dongbiao/GCN/data/synthetic_data/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/GCN/data/synthetic_data/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/GCN/data/synthetic_data/data/test_{n}.csv")
        
    n = n + 1

### Synthetic data suxiaoquan

In [641]:
table = pd.read_csv("/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/Synthetic_Dataset_1.csv")

In [643]:
metadata = pd.DataFrame({"sample_id": table.iloc[:, 0].values, "group": table["Group"].values})
metadata["group"] = [0 if i == "Group_S1_S3" else 1 for i in metadata["group"].values]
metadata.to_csv("/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/metadata.txt", sep="\t", index=False)

In [635]:
table = biom.Table(data=table.iloc[:, 1:61].values.T, observation_ids=table.iloc[:, 1:61].columns.values, sample_ids=table.iloc[:, 0].values)
with biom.util.biom_open("/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/table.biom", 'w') as f:
    table.to_hdf5(f, 'example')

In [638]:
metadata = metadata.set_index("sample_id")

In [639]:
table = biom.load_table("/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/table.biom")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
sid = table.ids(axis="sample")
n = 1
for train_idx, test_idx in kf.split(sid):
    table_train = table.filter(sid[train_idx], axis="sample", inplace=False)
    table_train.remove_empty()
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/data/train_{n}.biom", 'w') as f:
        table_train.to_hdf5(f, 'example')
    
    table_test = table.filter(sid[test_idx], axis="sample", inplace=False)
    table_test.remove_empty()
    with biom.util.biom_open(f"/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/data/test_{n}.biom", 'w') as f:
        table_test.to_hdf5(f, 'example')

    table_train = table_train.to_dataframe().T
    table_train.loc[:, "Group"] = metadata.loc[table_train.index.values, "group"]
    table_train.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/data/train_{n}.csv")
    table_test = table_test.to_dataframe().T
    table_test.loc[:, "Group"] = metadata.loc[table_test.index.values, "group"]
    table_test.to_csv(f"/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/data/test_{n}.csv")
        
    n = n + 1

In [644]:
tree_1 = TreeNode.read("/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/phylogeny.nwk")
tree_2 = TreeNode.read("/home/dongbiao/GCN/data/synthetic_data/phylogeny.nwk")

In [645]:
tree_1_distance = tree_1.tip_tip_distances()
tree_2_distance = tree_2.tip_tip_distances()

In [647]:
tree_1_distance.data

array([[0.     , 0.21668, 0.2518 , ..., 0.38665, 0.37041, 0.35561],
       [0.21668, 0.     , 0.22174, ..., 0.41933, 0.40309, 0.38829],
       [0.2518 , 0.22174, 0.     , ..., 0.45445, 0.43821, 0.42341],
       ...,
       [0.38665, 0.41933, 0.45445, ..., 0.     , 0.2272 , 0.24656],
       [0.37041, 0.40309, 0.43821, ..., 0.2272 , 0.     , 0.23032],
       [0.35561, 0.38829, 0.42341, ..., 0.24656, 0.23032, 0.     ]])

In [648]:
tree_2_distance.data

array([[  0.      ,  25.260833, 117.193324, ..., 332.71476 , 310.231344,
        294.66027 ],
       [ 25.260833,   0.      , 120.236223, ..., 335.757659, 313.274243,
        297.703169],
       [117.193324, 120.236223,   0.      , ..., 308.432318, 285.948902,
        270.377828],
       ...,
       [332.71476 , 335.757659, 308.432318, ...,   0.      , 199.988276,
        372.112816],
       [310.231344, 313.274243, 285.948902, ..., 199.988276,   0.      ,
        349.6294  ],
       [294.66027 , 297.703169, 270.377828, ..., 372.112816, 349.6294  ,
          0.      ]])

In [655]:
def gaussian_kernel(d, rho=2):
    """
    weight = exp(-2 * ρ * d^2)
    """
    return np.exp(-2 * rho * d ** 2)

In [665]:
gaussian_kernel(0.4, rho=5e-5)

0.9999840001279994

In [38]:
def normalize_tree_for_gnn(input_path, output_path):
    tree = TreeNode.read(input_path)
    max_depth = 0.0
    for tip in tree.tips():
        dist = tip.accumulate_to_ancestor(tree)
        if dist > max_depth:
            max_depth = dist

    count = 0
    for node in tree.traverse():
        if node.length is not None:
            node.length = node.length / max_depth
            count += 1

    tips = list(tree.tips())
    if len(tips) >= 2:
        d = tips[0].distance(tips[1])

    tree.write(output_path)

In [867]:
input_tree_file = "/home/dongbiao/GCN/data/synthetic_data/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/GCN/data/synthetic_data/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [805]:
input_tree_file = "/home/dongbiao/GCN/data/IBD/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/GCN/data/IBD/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [834]:
input_tree_file = "/home/dongbiao/GCN/data/CRC/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/GCN/data/CRC/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [854]:
input_tree_file = "/home/dongbiao/GCN/data/dietary_fiber/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/GCN/data/dietary_fiber/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [1055]:
input_tree_file = "/home/dongbiao/GCN/data/OSCC_16S/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/GCN/data/OSCC_16S/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [49]:
input_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/16S_CRC.nwk" 
output_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [282]:
input_tree_file = "/home/dongbiao/GCN/data/dietary_fiber/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/GCN/data/dietary_fiber/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [1019]:
input_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [1044]:
input_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

In [39]:
input_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/phylogeny.nwk" 
output_tree_file = "/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/phylogeny_normalize.nwk"
normalize_tree_for_gnn(input_tree_file, output_tree_file)

### Deepphylo Embedding

In [54]:
reducer = PCA(n_components=32)

In [57]:
### synthetic data
tree = TreeNode.read("/home/dongbiao/software/Phylo-Spec/data/Synthetic_Dataset_1/phylogeny.nwk")
dm = tree.tip_tip_distances()
dm = pd.DataFrame(data=dm.data, index=dm.ids, columns=dm.ids)
phy_embedding = reducer.fit_transform(dm.values)
phy_embedding = pd.DataFrame(data=phy_embedding, index=dm.index.values)
phy_embedding.to_csv("/home/dongbiao/GCN/data/synthetic_data/PCA_32.txt", sep="\t")

In [58]:
### IBD 16S
tree = TreeNode.read("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_IBD/phylogeny_normalize.nwk")
dm = tree.tip_tip_distances()
dm = pd.DataFrame(data=dm.data, index=dm.ids, columns=dm.ids)
phy_embedding = reducer.fit_transform(dm.values)
phy_embedding = pd.DataFrame(data=phy_embedding, index=dm.index.values)
phy_embedding.to_csv("/home/dongbiao/GCN/data/IBD_16S/PCA_32.txt", sep="\t")

In [59]:
### CRC 16S
tree = TreeNode.read("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_16S_CRC/phylogeny_normalize.nwk")
dm = tree.tip_tip_distances()
dm = pd.DataFrame(data=dm.data, index=dm.ids, columns=dm.ids)
phy_embedding = reducer.fit_transform(dm.values)
phy_embedding = pd.DataFrame(data=phy_embedding, index=dm.index.values)
phy_embedding.to_csv("/home/dongbiao/GCN/data/CRC_16S//PCA_32.txt", sep="\t")

In [283]:
### dietary fiber 16S
tree = TreeNode.read("/home/dongbiao/GCN/data/dietary_fiber/phylogeny_normalize.nwk")
dm = tree.tip_tip_distances()
dm = pd.DataFrame(data=dm.data, index=dm.ids, columns=dm.ids)
phy_embedding = reducer.fit_transform(dm.values)
phy_embedding = pd.DataFrame(data=phy_embedding, index=dm.index.values)
phy_embedding.to_csv("/home/dongbiao/GCN/data/dietary_fiber/PCA_32.txt", sep="\t")

In [60]:
### CRC WGC
tree = TreeNode.read("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_CRC/phylogeny_normalize.nwk")
dm = tree.tip_tip_distances()
dm = pd.DataFrame(data=dm.data, index=dm.ids, columns=dm.ids)
phy_embedding = reducer.fit_transform(dm.values)
phy_embedding = pd.DataFrame(data=phy_embedding, index=dm.index.values)
phy_embedding.to_csv("/home/dongbiao/GCN/data/CRC_WGS/PCA_32.txt", sep="\t")

In [61]:
### T2D WGS
tree = TreeNode.read("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_WGS_T2D/phylogeny_normalize.nwk")
dm = tree.tip_tip_distances()
dm = pd.DataFrame(data=dm.data, index=dm.ids, columns=dm.ids)
phy_embedding = reducer.fit_transform(dm.values)
phy_embedding = pd.DataFrame(data=phy_embedding, index=dm.index.values)
phy_embedding.to_csv("/home/dongbiao/GCN/data/T2D_WGS/PCA_32.txt", sep="\t")

In [62]:
### Multi-classification 16S
tree = TreeNode.read("/home/dongbiao/software/Phylo-Spec/data/Real_Dateset_Multi-classification/phylogeny_normalize.nwk")
dm = tree.tip_tip_distances()
dm = pd.DataFrame(data=dm.data, index=dm.ids, columns=dm.ids)
phy_embedding = reducer.fit_transform(dm.values)
phy_embedding = pd.DataFrame(data=phy_embedding, index=dm.index.values)
phy_embedding.to_csv("/home/dongbiao/GCN/data/Multi_classification/PCA_32.txt", sep="\t")